# Image Classification using Convolutional Neural Networks in PyTorch

In [1]:
# Importing required libraries

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms

In [3]:
# Downloading dataset

In [ ]:
transform = transforms.ToTensor()
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)


testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

 13%|████████▎                                                       | 22052864/170498071 [13:50<3:08:07, 13151.26it/s]

In [ ]:
target_classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')